Source : https://mediastack.com/

API gratuite : 500 requêtes par mois

S'inscrire pour avoir une clé API

Articles de presse.

# Clé API

In [3]:
from dotenv import load_dotenv
import os

In [15]:
API_KEY = os.getenv('CLE_API')
ABC = os.getenv('AUTRECHOSE')

#exemple : API_KEY = "123456789abcdefghi"

In [5]:
API_KEY

'a467a013b407850f200694a07dba3d63'

In [16]:
ABC

# Importations et définition du / des mots clés

## Mots clés

In [ ]:
mots_cles = "jeux"

## Importations

In [ ]:
pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=9e3b1efa8154fba8de42781df34f3961aadefd21c1f4ac8fea2859bb6dc85bf8
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from wordcloud import WordCloud
from stop_words import get_stop_words
import spacy
from collections import defaultdict

## Paramètres

In [ ]:
params = {
    "access_key": API_KEY,
    "keywords": mots_cles,
    "languages": "fr",
    "countries": "fr",
    "sources": "-franceantilles",
    "limit": 100,
    "offset": 0,
}

url = "http://api.mediastack.com/v1/news"


In [ ]:
# limit 100 et offset = 0 : prendre les articles de 0 à 99
# limit 100 et offset = 200 : prendre les articles de 200 à 299

# Récupération et création d'un tableau

In [ ]:
response = requests.get(url, params=params)

In [ ]:
data = response.json()

In [ ]:
data

{'pagination': {'limit': 100, 'offset': 0, 'count': 100, 'total': 192},
 'data': [{'author': None,
   'title': 'Budget 2025\xa0: face à la fronde des départements, Barnier calme le jeu',
   'description': "Les departements critiquaient l'obligation de tailler dans leurs finances pour participer a l'effort de reduction du deficit public demande par l'executif.",
   'url': 'https://www.lepoint.fr/politique/budget-2025-face-a-la-fronde-des-departements-barnier-calme-le-jeu-15-11-2024-2575375_20.php',
   'source': 'lepoint',
   'image': 'https://www.lepoint.fr/images/2024/11/15/27041294lpw-27041313-article-jpg_10646927.jpg',
   'category': 'general',
   'language': 'fr',
   'country': 'fr',
   'published_at': '2024-11-15T12:02:00+00:00'},
  {'author': 'LIBERATION, AFP',
   'title': '«Intervilles sans les vaches, ce n’est pas Intervilles» : Dax, Mont-de-Marsan et Bayonne ne veulent plus participer au jeu télévisé sans vachette',
   'description': 'Après Dax et Mont-de-Marsan, la ville de Ba

In [ ]:
all_articles = []

for _ in range(5):
    #Le nombre mis dans le range donnera le nombre de centaines d'articles
    #(mais aussi le nombre d'appels à l'API)
    response = requests.get(url, params=params)
    data = response.json()
    all_articles.extend(data["data"])
    params["offset"] += 100

#Création du dataframe
df = pd.DataFrame(all_articles)
df = df[['source', 'title', 'description', 'url', 'image', 'category', 'language', 'country', 'published_at']]

#Affichage de 3 lignes aléatoires
df.sample(3)

,source,title,description,url,image,category,language,country,published_at
9,Le Monde,"En direct, présidentielle américaine 2024 : Ka...","La candidate démocrate s’est targuée, jeudi à ...",https://www.lemonde.fr/international/live/2024...,https://img.lemde.fr/2024/10/24/852/0/6000/300...,general,fr,fr,2024-10-24T21:15:43+00:00
49,lepoint,Macron : le jeu qui rend fou,LE QUARTIER LIBRE DE SERGE RAFFY. Comment exas...,https://www.lepoint.fr/politique/50-jours-sans...,https://www.lepoint.fr/images/2024/09/04/26824...,general,fr,fr,2024-09-04T13:01:00+00:00
165,lepoint,Comment les JO ont contrarié les déclinistes,Les Jeux sont une reussite. Ils mettent en evi...,https://www.lepoint.fr/politique/comment-les-j...,https://www.lepoint.fr/images/2024/08/06/26766...,general,fr,fr,2024-08-06T12:07:00+00:00


# Premières statistiques

## Statistiques clés

In [ ]:
#Quelques stats
print(f"Nombre total d'articles: {len(df)}")
print(f"Nombre de sources uniques: {df['source'].nunique()}")
print(f"Nombre de pays uniques: {df['country'].nunique()}")
print(f"Nombre de catégories uniques: {df['category'].nunique()}")

#Conversion de la colonne published_at en date
df['published_at'] = pd.to_datetime(df['published_at'])

#Conversion de la date au format souhaité
oldest_date = df['published_at'].min().strftime('%d/%m/%Y')
newest_date = df['published_at'].max().strftime('%d/%m/%Y')

print(f"Article le plus ancien: {oldest_date}")
print(f"Article le plus récent: {newest_date}")

## MAJ des pays

In [ ]:

#Dictionnaire pour mapper les abréviations de pays aux noms complets
country_map = {
    'ar': 'Argentina', 'au': 'Australia', 'at': 'Austria', 'be': 'Belgium', 'br': 'Brazil', 'bg': 'Bulgaria',
    'ca': 'Canada', 'cn': 'China', 'co': 'Colombia', 'cz': 'Czech Republic', 'eg': 'Egypt', 'fr': 'France',
    'de': 'Germany', 'gr': 'Greece', 'hk': 'Hong Kong', 'hu': 'Hungary', 'in': 'India', 'id': 'Indonesia',
    'ie': 'Ireland', 'il': 'Israel', 'it': 'Italy', 'jp': 'Japan', 'lv': 'Latvia', 'lt': 'Lithuania',
    'my': 'Malaysia', 'mx': 'Mexico', 'ma': 'Morocco', 'nl': 'Netherlands', 'nz': 'New Zealand', 'ng': 'Nigeria',
    'no': 'Norway', 'ph': 'Philippines', 'pl': 'Poland', 'pt': 'Portugal', 'ro': 'Romania', 'sa': 'Saudi Arabia',
    'rs': 'Serbia', 'sg': 'Singapore', 'sk': 'Slovakia', 'si': 'Slovenia', 'za': 'South Africa', 'kr': 'South Korea',
    'se': 'Sweden', 'ch': 'Switzerland', 'tw': 'Taiwan', 'th': 'Thailand', 'tr': 'Turkey', 'ae': 'UAE',
    'ua': 'Ukraine', 'gb': 'United Kingdom', 'us': 'United States', 've': 'Venezuela'
}

#Remplacer les abréviations de pays par leur nom complet
df['country'] = df['country'].map(country_map)

#Supprimer les colonnes 'url' et 'image' du dataframe
#(qui ne servent pas à grand chose ici)
df = df.drop(['url', 'image'], axis=1)

#Affichage de 3 lignes aléatoires
df.sample(3)

## Nombre d'articles par source

In [ ]:
source_counts = df['source'].value_counts()

#Créer le diagramme en barres
plt.figure(figsize=(10, 6))
source_counts.plot(kind='bar')
plt.xlabel('Source')
plt.ylabel("Nombre d'articles")
plt.title("Nombre d'articles par source")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Nombre d'articles par pays

In [ ]:
source_counts = df['country'].value_counts()

#Créer le diagramme en barres
plt.figure(figsize=(10, 6))
source_counts.plot(kind='bar')
plt.xlabel('Pays')
plt.ylabel('Nombre d\'articles')
plt.title('Nombre d\'articles par pays')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Nombre d'articles par catégorie

In [ ]:
source_counts = df['category'].value_counts()

#Créer le diagramme en barres
plt.figure(figsize=(10, 6))
source_counts.plot(kind='bar')
plt.xlabel('Catégories')
plt.ylabel('Nombre d\'articles')
plt.title('Nombre d\'articles par catégorie')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
df['category'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.title('Répartition des articles par catégorie')
plt.show()

# Nuage de mots

In [ ]:
#Concaténer tous les titres et toutes les descriptions
all_titles = ' '.join(df['title'])
all_descriptions = ' '.join(df['description'])

## Sans stopwords

In [ ]:
#Créer les nuages de mots
wordcloud_titles = WordCloud(width=800, height=400).generate(all_titles)
wordcloud_descriptions = WordCloud(width=800, height=400).generate(all_descriptions)
wordcloud_combined = WordCloud(width=800, height=400).generate(all_titles + ' ' + all_descriptions)

#Afficher les nuages de mots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

ax1.imshow(wordcloud_titles, interpolation='bilinear')
ax1.set_title('Nuage de mots (Titres)')
ax1.axis('off')

ax2.imshow(wordcloud_descriptions, interpolation='bilinear')
ax2.set_title('Nuage de mots (Descriptions)')
ax2.axis('off')

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud_combined, interpolation='bilinear')
plt.title('Nuage de mots (Titres + Descriptions)')
plt.axis('off')

plt.show()

## Avec stopwords

In [ ]:
#Ajout de stopwords personnalisés
stopwords_persos = {mots_cles}

In [ ]:
#Stopwords français
stopwords = set(get_stop_words('fr'))
print(stopwords)

In [ ]:
stopwords = set(get_stop_words('fr'))

#Ajouter des mots vides supplémentaires
additional_stopwords = {
    'alors', 'an', 'ans', 'après', 'au', 'aucuns', 'aussi', 'autre', 'avant', 'avec', 'avoir', 'bon', 'car', 'ce', 'cela', 'ces', 'ceux',
    'chaque', 'ci', 'comme', 'comment', 'dans', 'des', 'du', 'dedans', 'dehors', 'depuis', 'devrait', 'doit', 'donc',
    'dos', 'début', 'elle', 'elles', 'en', 'encore', 'essai', 'est', 'et', 'eu', 'fait', 'faites', 'femme' 'fois', 'font', 'homme',
    'hors', 'ici', 'il', 'ils', 'je', 'juste', 'la', 'le', 'les', 'leur', 'là', 'ma', 'maintenant', 'mais', 'mes',
    'mien', 'moins', 'mon', 'mot', 'même', 'ni', 'nommés', 'notre', 'nous', 'ou', 'où', 'par', 'parce', 'pas', 'peut',
    'peu', 'plupart', 'pour', 'pendant', 'pourquoi', 'quand', 'que', 'quel', 'quelle', 'quelles', 'quels', 'qui', 'sa', 'sans',
    'ses', 'seulement', 'si', 'sien', 'son', 'sont', 'sous', 'soyez', 'sujet', 'sur', 'ta', 'tandis', 'tellement',
    'tels', 'tes', 'ton', 'tous', 'tout', 'trop', 'très', 'tu', 'voient', 'vont', 'votre', 'vous', 'vu', 'ça', 'étaient',
    'état', 'étions', 'été', 'être', 's', "s'", 'plus',
    'afin', 'ainsi', 'allaient', 'allait', 'allons', 'assez', 'aucun', 'aucune', 'auprès', 'auquel', 'auquelles', 'auquels',
    'aussitôt', 'autant', 'autour', 'autrefois', 'autrement', 'autrui', 'avaient', 'avait', 'beaucoup', 'bien', 'bigre',
    'bougre', 'boum', 'bravo', 'brrr', 'c', 'cent', 'cependant', 'certain', 'certaine', 'certaines', 'certains', 'certes',
    'chez', 'chiche', 'chut', 'contre', 'couic', 'crac', 'd', 'da', 'dans', 'de', 'debout', 'deça', 'delà', 'derrière',
    'désormais', 'dessous', 'dessus', 'deux', 'devant', 'devers', 'divers', 'diverse', 'diverses', 'dix', 'dès', 'en', 'entre',
    'envers', 'environ', 'et', 'etc', 'fais', 'faisaient', 'faisions', 'feront', 'flac', 'fors', 'hormis', 'houp', 'hui', 'huit',
    'hé', 'hélas', 'jusqu', 'jusque', 'l', 'laquelle', 'lesquelles', 'lesquels', 'leurs', 'lez', 'lors', 'lorsque', 'lui', 'lès',
    'm', 'maint', 'mainte', 'maintes', 'maints', 'malgré', 'me', 'merci', 'mes', 'mien', 'mienne', 'miennes', 'miens', 'mille',
    'mince', 'moyennant', 'na', 'ne', 'neuf', 'néanmoins', 'nôtre', 'nôtres', 'oh', 'ollé', 'olé', 'on', 'ont', 'ouf', 'ouias',
    'oust', 'ouste', 'outre', 'pan', 'par-dessus', 'pardi', 'partant', 'plein', 'plusieurs', 'plus', 'pour', 'pourtant', 'pouah',
    'puis', 'puisque', 'qu', 'quant', 'quatre', 'quelconque', 'quelque', 'quelques', 'quiconque', 'quoi', 'quoique', 'revoici',
    'revoilà', 'rien', 's', 'sacrebleu', 'sans', 'sauf', 'se', 'seize', 'selon', 'sept', 'si', 'sien', 'sienne', 'siennes', 'siens',
    'sitôt', 'six', 'soi', 'sommes', 'son', 'sus', 't', 'tacatac', 'te', 'tien', 'tienne', 'tiennes', 'tiens', 'toc', 'toi', 'ton',
    'toujours', 'tous', 'tout', 'toute', 'toutes', 'treize', 'trente', 'trois', 'tsoin', 'tsouin', 'tu', 'un', 'une', 'vingt', 'voici',
    'voilà', 'vos', 'vôtre', 'vôtres', 'zut'
}

#Regroupement
stopwords = stopwords.union(stopwords_persos)
stopwords = stopwords.union(additional_stopwords)


#Créer les nuages de mots en excluant les mots vides
wordcloud_titles = WordCloud(width=800, height=400, stopwords=stopwords).generate(all_titles)
wordcloud_descriptions = WordCloud(width=800, height=400, stopwords=stopwords).generate(all_descriptions)
wordcloud_combined = WordCloud(width=800, height=400, stopwords=stopwords).generate(all_titles + ' ' + all_descriptions)

#Afficher les nuages de mots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
ax1.imshow(wordcloud_titles, interpolation='bilinear')
ax1.set_title('Nuage de mots (Titres)')
ax1.axis('off')
ax2.imshow(wordcloud_descriptions, interpolation='bilinear')
ax2.set_title('Nuage de mots (Descriptions)')
ax2.axis('off')

plt.figure(figsize=(20, 10))
plt.imshow(wordcloud_combined, interpolation='bilinear')
plt.title('Nuage de mots (Titres + Descriptions)')
plt.axis('off')
plt.show()

# Petite analyse temporelle

In [ ]:
#Convertir la colonne 'published_at' en format datetime
df['published_at'] = pd.to_datetime(df['published_at'])

#Compter le nombre d'articles par jour
articles_par_jour = df.resample('D', on='published_at').size()

#Tracer le graphique
plt.figure(figsize=(12, 6))
articles_par_jour.plot(kind='line')
plt.title('Nombre d\'articles publiés par jour')
plt.xlabel('Date')
plt.ylabel('Nombre d\'articles')
plt.show()

# Analyse des entités nommées

In [ ]:
#En français
nlp = spacy.load('fr_core_news_lg')

#Extraire les entités nommées d'un texte donné à l'aide de spaCy
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents if len(ent.text) > 2]

#Mise dans le dataframe
df['entities'] = df['description'].apply(extract_entities)

#Affichage de 3 lignes aléatoires
df.sample(3)

In [ ]:
#Affichage des entités nommées les plus fréquentes
all_entities = [ent for sublist in df['entities'] for ent in sublist]
print(f"Entités nommées les plus fréquentes:")
for ent, count in pd.Series(all_entities).value_counts().head(10).items():
    print(f"{ent[0]} ({ent[1]}): {count}")

# Analyse des émotions

In [ ]:
#Dictionnaire contenant les mots-clés pour chaque émotion
emotion_keywords = {
    'peur': ['peur', 'crainte', 'inquiétude', 'angoisse', 'terreur', 'effroi', 'frayeur', 'panique', 'anxiété', 'appréhension', 'hantise', 'phobies', 'cauchemar', 'frisson', 'tremblement', 'alarme', 'trac', 'timidité', 'lâcheté', 'froussardise', 'couardise', 'pusillanimité', 'menace', 'péril', 'danger', 'risque', 'incertitude', 'doute', 'hésitation', 'stress', 'tension', 'nervosité', 'affolement', 'agitation', 'trouble', 'tourment', 'tracas', 'souci', 'apeuré', 'craintif', 'effrayé', 'terrorisé', 'épouvanté', 'paniqué', 'anxieux', 'inquiet', 'alarmé', 'oppressé', 'pétrifié', 'paralysé', 'intimidé', 'affolé', 'horrifié', 'tétanisé', 'angoissé', 'stressé', 'tendu', 'nerveux', 'préoccupé', 'obsédé', 'hanté', 'traumatisé', 'choqué', 'bouleversé', 'perturbé', 'désorienté', 'déstabilisé', 'ébranlé', 'secoué', 'alerte', 'vigilant', 'méfiant', 'suspicieux', 'paranoïaque', 'phobie', 'agoraphobie', 'claustrophobie', 'acrophobie', 'arachnophobie', 'ophiophobie', 'cynophobie', 'trypanophobie', 'thanatophobie', 'nosophobie', 'bélénophobie', 'sidérodromophobie'],

    'espoir': ['espoir', 'espérance', 'optimisme', 'confiance', 'foi', 'attente', 'croyance', 'aspiration', 'rêve', 'illusion', 'utopie', 'idéal', 'désir', 'souhait', 'vœu', 'ambition', 'perspective', 'avenir', 'projection', 'promesse', 'lueur', 'possibilité', 'chance', 'opportunité', 'éventualité', 'expectative', 'anticipation', 'supposition', 'présomption', 'conviction', 'persuasion', 'assurance', 'certitude', 'évidence', 'encouragement', 'réconfort', 'soulagement', 'consolation', 'réjouissance', 'allégresse', 'jubilation', 'exultation', 'engouement', 'enthousiasme', 'ardeur', 'ferveur', 'passion', 'zèle', 'optimiste', 'confiant', 'enthousiaste', 'positif', 'convaincu', 'persuadé', 'assuré', 'certain', 'sûr', 'déterminé', 'résolu', 'décidé', 'audacieux', 'intrépide', 'courageux', 'vaillant', 'héroïque', 'brave', 'hardi', 'téméraire', 'aventureux', 'entreprenant', 'ambitieux', 'conquérant', 'dynamique', 'énergique', 'volontaire', 'tenace', 'persévérant', 'obstiné', 'acharné', 'inébranlable', 'inflexible', 'indomptable', 'invincible', 'imbattable', 'indestructible', 'immortel', 'prophétique'],

    'joie': ['joie', 'bonheur', 'plaisir', 'allégresse', 'gaieté', 'euphorie', 'ravissement', 'béatitude', 'félicité', 'extase', 'jubilation', 'enchantement', 'enthousiasme', 'exultation', 'transport', 'ivresse', 'délire', 'satisfaction', 'contentement', 'épanouissement', 'épanouissement', 'rayonnement', 'exaltation', 'effervescence', 'animation', 'vivacité', 'dynamisme', 'énergie', 'fougue', 'ardeur', 'ferveur', 'passion', 'engouement', 'emballement', 'ébullition', 'bouillonnement', 'débordement', 'déferlement', 'liesse', 'délectation', 'réjouissance', 'régal', 'festin', 'fête', 'célébration', 'apothéose', 'triomphe', 'victoire', 'gloire', 'splendeur', 'magnificence', 'somptuosité', 'hilarité', 'rire', 'sourire', 'éclat de rire', 'gloussement', 'ricanement', 'fou rire', 'amusement', 'divertissement', 'récréation', 'agrément', 'jouissance', 'volupté', 'sensualité', 'érotisme', 'orgasme', 'excitation', 'griserie', 'vertige', 'fascination', 'envoûtement', 'émerveillement', 'éblouissement', 'émerveillement', 'admiration', 'adoration', 'vénération', 'idolâtrie', 'dévotion', 'gratitude', 'reconnaissance', 'grâce', 'bénédiction', 'miracle', 'prodige', 'merveille', "chef-d'œuvre", 'trésor', 'bijou', 'perle', 'joyau', 'délice'],

    'colère': ['colère', 'rage', 'fureur', 'furie', 'courroux', 'exaspération', 'indignation', 'irritation', 'agacement', 'énervement', 'crispation', 'hargne', 'agressivité', 'violence', 'brutalité', 'frénésie', 'hystérie', 'excitation', 'emportement', 'déchaînement', 'véhémence', 'virulence', 'animosité', 'hostilité', 'ressentiment', 'rancune', 'rancœur', 'amertume', 'aigreur', 'acrimonie', 'haine', 'détestation', 'aversion', 'répulsion', 'dégoût', 'écœurement', 'répugnance', 'agressif', 'violent', 'brutal', 'féroce', 'sauvage', 'cruel', 'impitoyable', 'impétueux', 'irascible', 'colérique', 'bilieux', 'acariâtre', 'irritable', 'susceptible', 'ombrageux', 'belliqueux', 'querelleur', 'bagarreur', 'hargneux', 'haineux', 'malveillant', 'malintentionné', 'malfaisant', 'nuisible', 'nocif', 'toxique', 'venimeux', 'méchant', 'vicieux', 'ignoble', 'abject', 'odieux', 'révoltant', 'scandaleux', 'offensant', 'blessant', 'injurieux', 'outrageant', 'diffamatoire', 'calomnieux', 'infamant', 'dégradant', 'humiliant', 'mortifiant', 'vexant', 'frustrant', 'contrariété'],

    'tristesse': ['tristesse', 'chagrin', 'peine', 'douleur', 'souffrance', 'affliction', 'abattement', 'accablement', 'désespoir', 'désolation', 'amertume', 'mélancolie', 'nostalgie', 'spleen', 'cafard', 'morosité', 'déprime', 'dépression', 'neurasthénie', 'langueur', 'découragement', 'lassitude', 'lamento', 'apitoiement', 'déploration', 'jérémiade', 'complainte', 'lamentation', 'litanie', 'thrène', 'élégie', 'oraison funèbre', 'nénies', 'regret', 'remords', 'repentance', 'repentir', 'componction', 'contrition', 'pénitence', 'expiation', 'rédemption', 'rachat', 'purgatoire', 'enfer', 'damnation', 'tourment', 'supplice', 'martyre', 'torture', 'crucifixion', 'mortification', 'macération', 'cilice', 'haire', 'flagellation', 'malheur', 'infortune', 'adversité', 'calamité', 'catastrophe', 'désastre', 'fléau', 'malheureux', 'infortuné', 'misérable', 'pitoyable', 'pathétique', 'lamentable', 'déplorable', 'navrant', 'poignant', 'déchirant', 'bouleversant', 'accablant', 'affligeant', 'démoralisant', 'déprimant', 'éprouvant', 'éreintant', 'épuisant', 'exténuant', 'tuant', 'mortel', 'fatal', 'funeste', 'sinistre', 'lugubre', 'funèbre', 'macabre', 'mortuaire', 'sépulcral', 'morbide', 'nécrophile']
}

#Fonction pour compter les occurrences des mots-clés d'émotions dans un texte donné
def count_emotions(text, keywords):
    counts = {emotion: 0 for emotion in keywords}
    for word in text.split():
        for emotion, kws in keywords.items():
            if word.lower() in kws:
                counts[emotion] += 1
    return counts

#Appliquer la fonction count_emotions aux titres et descriptions des articles
df['emotion_counts_titles'] = df['title'].apply(lambda x: count_emotions(x, emotion_keywords))
df['emotion_counts_descriptions'] = df['description'].apply(lambda x: count_emotions(x, emotion_keywords))

#Calculer les totaux d'émotions pour les titres et les descriptions
title_emotion_totals = df['emotion_counts_titles'].apply(pd.Series).sum()
description_emotion_totals = df['emotion_counts_descriptions'].apply(pd.Series).sum()

#Combiner les totaux d'émotions des titres et des descriptions
combined_emotion_totals = title_emotion_totals + description_emotion_totals

#Affichage de 3 lignes aléatoires
df.sample(3)

In [ ]:
plt.figure(figsize=(12, 8))

#Graphique 1 : Émotions dans les titres des articles
plt.subplot(2, 2, 1)
title_emotion_totals.plot(kind='bar')
plt.title('Émotions dans les titres des articles')
plt.xlabel('Émotion')
plt.ylabel('Nombre d\'occurrences')

#Graphique 2 : Émotions dans les descriptions des articles
plt.subplot(2, 2, 2)
description_emotion_totals.plot(kind='bar')
plt.title('Émotions dans les descriptions des articles')
plt.xlabel('Émotion')
plt.ylabel('Nombre d\'occurrences')

#Graphique 3 : Émotions dans les titres et descriptions combinés
plt.subplot(2, 1, 2)
combined_emotion_totals.plot(kind='bar')
plt.title('Émotions dans les titres et descriptions combinés')
plt.xlabel('Émotion')
plt.ylabel('Nombre d\'occurrences')

plt.tight_layout()
plt.show()

# Analyse des co-occurences de mots

In [ ]:
def cooccurrence(texts, keyword, stopwords):
    cooc = defaultdict(int)
    for text in texts:
        if keyword in text:
            for word in text.split():
                if word.lower() != keyword.lower() and word.lower() not in stopwords:
                    cooc[word] += 1
    return sorted(cooc.items(), key=lambda x: x[1], reverse=True)

# Exemple avec mots_cles
coocs = cooccurrence(df['title'], mots_cles, stopwords)
print(f"Mots apparaissant fréquemment avec {mots_cles}:")
for word, count in coocs[:10]:
    print(f"{word}: {count}")